In [1]:
ans_file=""
text=""
question=""

In [2]:
# Parameters
text = "Stoicism is a school of Hellenistic philosophy founded by Zeno of Citium in Athens in the early 3rd century BCE. It is a philosophy of personal virtue ethics informed by its system of logic and its views on the natural world, asserting that the practice of virtue is both necessary and sufficient to achieve eudaimonia (happiness, lit.\u2009'good spiritedness'): one flourishes by living an ethical life. The Stoics identified the path to eudaimonia with a life spent practicing virtue and living in accordance with nature.Alongside Aristotelian ethics, the Stoic tradition forms one of the major founding approaches to virtue ethics.[1] The Stoics are especially known for teaching that virtue is the only good for human beings, and that external things, such as health, wealth, and pleasure, are not good or bad in themselves (adiaphora) but have value as material for virtue to act upon. Many Stoics\u2014such as Seneca and Epictetus\u2014emphasized that because virtue is sufficient for happiness, a sage would be emotionally resilient to misfortune. The Stoics also held that certain destructive emotions resulted from errors of judgment, and they believed people should aim to maintain a will (called prohairesis) that is in accordance with nature. Because of this, the Stoics thought the best indication of an individual's philosophy was not what a person said but how a person behaved.[2] To live a good life, one had to understand the rules of the natural order since they believed everything was rooted in nature."
question = "When was Stoicism started? "
ans_file = "QAS_ans.txt"


In [3]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import re
import time
import papermill as pm
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Loading Model and Encoder
model=torch.load("MODEL_APR_23")
model.eval()
tokenizer = torch.load("tokenizer_encoder.unknown")

In [5]:
def question_answer_A(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)

    #number of tokens in segment A - question
    num_seg_a = sep_idx+1

    #number of tokens in segment B - text
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    answer = 'unable to find answer'
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    return torch.max(output.start_logits), answer.capitalize() #score ans

In [6]:
def chunk_text(text, max_len):
    # split the text into sentences
    sentences = re.findall('[^.!?]+[.!?]', text)
    
    # initialize variables
    current_len = 0
    current_chunk = []
    chunks = []
    
    # iterate over each sentence
    for sentence in sentences:
        # add the sentence to the current chunk
        current_chunk.append(sentence)
        
        # update the length of the current chunk
        current_len += len(sentence)
        
        # if the length of the current chunk is greater than or equal to the max length
        if current_len >= max_len:
            # add the current chunk to the list of chunks
            chunks.append(' '.join(current_chunk))
            
            # reset the current chunk and length
            current_chunk = []
            current_len = 0
    
    # add any remaining sentences as a final chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

In [7]:
def get_better_ans(question, text):
  chunk_of_text=chunk_text(text,450)
  scores = {}
  for i in chunk_of_text:
    scores.update({question_answer_A(question, i)[0] : question_answer_A(question, i)[1] })
  v = list(scores.values())
  k = list(scores.keys())
  return (v[k.index(max(k))])

In [8]:
print(text)
print(question)
x = get_better_ans(question, text)


Stoicism is a school of Hellenistic philosophy founded by Zeno of Citium in Athens in the early 3rd century BCE. It is a philosophy of personal virtue ethics informed by its system of logic and its views on the natural world, asserting that the practice of virtue is both necessary and sufficient to achieve eudaimonia (happiness, lit. 'good spiritedness'): one flourishes by living an ethical life. The Stoics identified the path to eudaimonia with a life spent practicing virtue and living in accordance with nature.Alongside Aristotelian ethics, the Stoic tradition forms one of the major founding approaches to virtue ethics.[1] The Stoics are especially known for teaching that virtue is the only good for human beings, and that external things, such as health, wealth, and pleasure, are not good or bad in themselves (adiaphora) but have value as material for virtue to act upon. Many Stoics—such as Seneca and Epictetus—emphasized that because virtue is sufficient for happiness, a sage would 

In [9]:
print(x)
f = open(ans_file, "w")
f.write(x)
f.close()

Early 3rd century bce
